In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("Max_Temp")
sc = SparkContext(conf = conf)
sc.setLogLevel("WARN")

21/11/29 08:16:30 WARN Utils: Your hostname, duyvan-Inspiron-5402 resolves to a loopback address: 127.0.1.1; using 10.0.130.75 instead (on interface wlp44s0)
21/11/29 08:16:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/29 08:16:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
def parseLine(line):
    fields = line.split(',')
    stationID = fields[0]
    entryType = fields[2]
    temperature = float(fields[3]) * 0.1 * (9.0 / 5.0) + 32.0
    return (stationID, entryType, temperature)

In [3]:
lines = sc.textFile("file:///home/duyvan/Documents/learning_spark/Section_2/1800.csv")
parsedLines = lines.map(parseLine)

In [4]:
print(parsedLines.take(5))

[('ITE00100554', 'TMAX', 18.5), ('ITE00100554', 'TMIN', 5.359999999999999), ('GM000010962', 'PRCP', 32.0), ('EZE00100082', 'TMAX', 16.52), ('EZE00100082', 'TMIN', 7.699999999999999)]


In [5]:
minTemps = parsedLines.filter(lambda x: "TMAX" in x[1])

In [6]:
stationTemps = minTemps.map(lambda x: (x[0], x[2]))

In [7]:
minTemps = stationTemps.reduceByKey(lambda x, y: max(x,y))
results = minTemps.collect();

for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))

ITE00100554	90.14F
EZE00100082	90.14F
